In [46]:
import delta_sharing
import pandas as pd
import geopandas
import json
import h3

from urllib.request import urlopen, urlretrieve
# Point to the profile file. It can be a file on the local file system or a file on a remote storage.
profile_file = "Kontur_Population_Global_H3_res_8_2021.share"

# Create a SharingClient.
client = delta_sharing.SharingClient(profile_file)

# List all shared tables.
shares = client.list_shares()
for share in shares:
  schemas = client.list_schemas(share)
  for schema in schemas:
    tables = client.list_tables(schema)
    for table in tables:
      print(f'name = {table.name}, share = {table.share}, schema = {table.schema}')
# Create a url to access a shared table.
# A table path is the profile file path following with `#` and the fully qualified name of a table 
# (`<share-name>.<schema-name>.<table-name>`).
# table_url = profile_file + "carto_demographics_population_glo_h3res8_v1_yearly_2022.carto.demographics_population_glo_h3res8_v1_yearly_2022"

# Fetch 10 rows from a table and convert it to a Pandas DataFrame. This can be used to read sample data 
# from a table that cannot fit in the memory.
# delta_sharing.load_as_pandas(table_url, limit=10)

name = demographics_population_glo_h3res8_v1_yearly_2022, share = carto_demographics_population_glo_h3res8_v1_yearly_2022, schema = carto


In [47]:
share_name = 'carto_demographics_population_glo_h3res8_v1_yearly_2022'
schema = 'carto'
table_name = 'demographics_population_glo_h3res8_v1_yearly_2022'
table_url = f'''{profile_file}#{share_name}.{schema}.{table_name}'''
df = delta_sharing.load_as_pandas(table_url)
world_pop = df.set_index('h3')
world_pop.head(20)

,do_date,population
h3,,
88266c1b05fffff,2022-01-01,256
882672c309fffff,2022-01-01,256
8826c844b1fffff,2022-01-01,256
8826c1b32dfffff,2022-01-01,256
8826cb934dfffff,2022-01-01,256
882671c189fffff,2022-01-01,256
8826c80e05fffff,2022-01-01,256
88266ac453fffff,2022-01-01,256
88269b1583fffff,2022-01-01,256


In [48]:
resolution = 8 
def polyfill_geo_json(geojson, res):
    out = h3.polyfill(geojson, res, True)
    
    return out

url = 'https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/country_shapes/exports/geojson'
jsonData = urlopen(url).read().decode('utf-8')
data = geopandas.read_file(jsonData, where="cntry_name='Vietnam'")
data = data.explode(index_parts=True)
gjson = data.to_json()
jsonObject = json.loads(gjson)
df = pd.DataFrame()
for i in jsonObject['features']:
    r = pd.DataFrame(polyfill_geo_json(i['geometry'], resolution), columns=['h3_hex_id'])
    df = pd.concat([df, r])
df['h3_resolution'] = resolution
country_h3 = df.set_index('h3_hex_id')
country_h3

,h3_resolution
h3_hex_id,
8865b405e9fffff,8
884143719dfffff,8
8841617107fffff,8
8840642b33fffff,8
884161132dfffff,8
...,...
8865858685fffff,8
88658580b7fffff,8
8865858481fffff,8


In [49]:
result = world_pop.merge(country_h3, how='inner', left_index=True, right_index=True)

In [54]:
result = result.h3.h3_to_geo()
result.head(20)

AttributeError: 'DataFrame' object has no attribute 'h3'

In [ ]:
    df = df.h3.geo_to_h3(resolution, lat_col = 'latitude', lng_col = 'longitude')
    df = df.h3.h3_to_geo()
    df = df.drop(columns='geometry')
    df.index.names = ['h3_hex_id']
    df = df.merge(country_h3, how='inner', left_index=True, right_index=True)
    if df.empty:
        print('DataFrame is empty!')
    else:
        write_deltalake("./delta/high_density_population_maps_data", df, mode="overwrite", partition_by=['filename'], partition_filters=[('filename', '=', filename)])
        
    # df['geocode_reverse'] = rg.search(df['h3_06_centroid'].values.tolist())
    # df['country'] = df['geocode_reverse'].apply(lambda x: x['cc'])
    # df['province'] = df['geocode_reverse'].apply(lambda x: x['admin1'])
    # df = df[df['country'] == 'VN']